<a href="https://colab.research.google.com/github/wahyunh10/Car-Auction-Prices-Project/blob/main/Car_Auction_Prices_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Auction Prices Project**

1. Role: Sebagai tim data scientist dalam suatu perusahaan bernama PT.Dats Your Future yang berperan sebagai dealer jual beli mobil bekas yang bertanggung jawab untuk memberikan rekomendasi harga mobil bekas berdasarkan data histori pembelian 2 tahun sebelumnya yang ada di Amerika yang diasumsikan ideal karena mobil berhasil dibeli dan dijual kembali ke konsumen dengan harga ideal.
2. Masalah:
  * Perusahaan menentukan harga beli mobil bekas tidak terstandar (tidak sesuai kondisi mobil sebenarnya) sehingga mendapat harga mobil bekas yang berbeda dari perspektif yang bias oleh setiap staff.
   * Proses screening dan negosiasi yang cukup lama akhirnya customer menjadi ragu dan cenderung tidak menjual mobil nya ke perusahaan kita.
3. Goal: Bisa membantu perusahaan menentukan harga beli mobil bekas yang dapat terstandarkan berdasarkan model machine learning.
4. Objective: Membuat tools / model machine learning untuk membantu memprediksi harga mobil berdasarkan kondisi sebenarnya.
5. usiness Metrics: Sales Pembelian dan Efisiensi Waktu

# **Final Project - EDA, Insight, Visualization**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# **Mengubah parameter default matplotlib**
Parameter default matplotlib dapat diubah dengan rcParams sebagai berikut

In [ ]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

# **Load dataset**
Kita akan melihat dataset prediksi harga mobil

The dataset contains historical car auction sales prices, scraped from the outside internet sources collected in 2015. This dataset taken from kaggle (https://www.kaggle.com/tunguz/used-car-auction-prices).

This dataset contains 491641 rows and 15 columns:

* year : information of year production of the car
* make : brand of the car
* model : model of the brand car
* trim : spesific model of the car
* body : body type of the car
* transmission : transmission type of the car
* vin : unique id of the car
* state : code state where the transaction happened
* condition : rating condition of the car
* odometer : total distance of the car
* color : body color of the car
* interior : interior color of the car
* seller : place that sold the car
* mmr : market price of the car
* sellingprice : price of the car
* saledate : date when the car sold

In [ ]:
df = pd.read_csv("car_prices.csv")
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,Unnamed: 16
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,financial services remarketing (lease),31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),NaN
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,4.1,14282.0,white,black,volvo na rep/world omni,27500,27750,Thu Jan 29 2015 04:30:00 GMT-0800 (PST),NaN
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,financial services remarketing (lease),66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST),NaN


# **Descriptive Statistics**

Hal pertama yang biasanya dilakukan dalam EDA adalah mengecek tipe data setiap kolom dan jumlah nilai yang hilang. Hal ini bisa kita lakukan dengan menggunakan df.info().

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493484 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547043 non-null  object 
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558837 non-null  object 
 14  sellingprice  558837 non-null  int64  
 15  saledate      558837 non-null  object 
 16  Unnamed: 16   25 non-null      object 
dtypes: float64(1), int64(2), object(14)
memory usage

Kita dapat melihat bahwa:

1. Dataframe memiliki total 558811 baris dan 16 kolom
2. Target output sepertinya adalah kolom `sellingprice` dengan tipe data `int64`
3. Berarti sisanya adalah feature
Dari informasi di atas, kita bisa pisahkan kolom kategorical dan numerical sebagai berikut:

Dari informasi di atas, kita bisa pisahkan kolom kategorical dan numerical sebagai berikut:

In [ ]:
cats = ['make', 'model', 'trim', 'body', 'transmission','vin','state','color','interior','seller','saledate']
nums = ['year', 'condition', 'odometer', 'mmr', 'sellingprice']

# **Sampling untuk memahami data**

Setelah mengetahui kolom apa saja yang ada di dalam dataset, kita lakukan sampling untuk memastikan apakah isi kolomnya sesuai ekspektasi. Biasakan lakukan ini beberapa kali karena seringkali apabila ada keanehan tidak akan keluar pada sampling pertama.

Sampling dapat dilakukan dengan menggunakan `df.sample()`

In [ ]:
df.sample(10)

Beberapa pengamatan:

* Target kita, `sellingprice` benar bertipe int64

# **Statistical summary**

Selanjutnya kita munculkan ringkasan statistik dataframe, baik untuk numerical dan categorical. Hal ini dilakukan untuk mengecek secara singkat akan keberadaan outlier dan karakteristik distribusi untuk feature numeric.

In [ ]:
df[nums].describe()

In [ ]:
df[cats].describe()

Beberapa pengamatan dari *statistical summary* kolom kategorikal di atas:

* Kolom `trim`,`vin`,`seller` memiliki jumlah nilai unik yang banyak, ada kemungkinan tidak terlalu berguna untuk modeling
* Mayoritas dari data set ini yaitu menjual Mobil dari pabrikan (`make`) Ford, dengan model (`body`) sedan.

# **Value Counting**

Kita dapat memastikan pengamatan mengenai kolom-kolom `categorical` di atas dengan value counting

In [ ]:
for col in cats:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

**Duplicates and Unique**

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
len(df.make.unique())

In [ ]:
# lowercase data make 
df.make = df.make.str.lower()
len(df.make.unique())

In [ ]:
plt.figure(figsize=(12,5))
chart = sns.countplot(x=df.make)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90);

> Berdasarkan data diatas ditemukan insight, bahwasanya mobil dengan merek Ford menjadi mobil yang paling sering dijual



In [ ]:
len(df.model.unique())

In [ ]:
# lowercase data make 
df.model = df.model.str.lower()
len(df.model.unique())

In [ ]:
df.model.value_counts()

In [ ]:
len(df.body.unique())

In [ ]:
df.body.unique()

In [ ]:
# lowercase data body 
df.body = df.body.str.lower()
len(df.body.unique())

In [ ]:
df.body.value_counts().head(10)

Beberapa pengamatan pada proses diatas:

* Tidak ada data yang bersifat duplikat
* Beberapa data pada kolom `make` diubah menjadi lowercase sehingga jumlah data unique dari 97 berubah menajadi 67
* Beberapa data pada kolom `model` diubah menjadi lowercase sehingga jumlah data unique dari 974 berubah menajadi 852
* Beberapa data pada kolom `make` diubah menjadi lowercase sehingga jumlah data unique dari 87 berubah menjadi 46

# **Univariate Analysis**

Setelah melakukan analisis sederhana tentang statistik deskriptif, sekarang kita fokus pada satu-persatu kolom dengan *Univariate Analysis*

# **Box plots**

In [ ]:
for i in range(0, len(nums)):
    plt.subplot(1, len(nums), i+1)
    sns.boxplot(y=df[nums[i]], color='gray', orient='v')
    plt.tight_layout()

Untuk boxplot, hal paling penting yang harus kita perhatikan adalah keberadaan outlier.

* Outlier terlihat hampir di setiap kolom kecuali condition
* Dari boxplotnya terlihat mana distribusi yang terlihat skewed: year, odometer, mmr,sellingprice

Kesimpulan:

* Mobil keluaran tahun 1997 ke bawah dianggap data outlier.
* Berdasarkan kondisi mobil, tidak ada yg termasuk data outlier
* Mobil dgn total perjalanan lebih dari 2000 km termasuk data outlier.
* Mobil dgn mmr lebih dari 35000 termasuk data outlier.
* Mobil dgn harga jual lebih dari 35000 termasuk data outlier.

# **Dist plots**

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(0, len(nums)):
    plt.subplot(3, len(nums)/2, i+1)
    sns.distplot(df[nums[i]], color='gray')
    plt.tight_layout()

Untuk distribution plot, hal utama yang perlu diperhatikan adalah bentuk distribusi:

* Seperti dugaan kita ketika melihat boxplot di atas, kolom `year`, `odometer`, `mmr`, `sellingpric`e *skewed*

Kesimpulan:

* Berarti ada kemungkinan kita perlu melakukan sesuatu pada kolom2 tersebut nantinya
* Selain kolom2 itu distribusinya terlihat normal-normal saja
* Mayoritas tahun pembuatan mobil ada pada rentang 2007-2013. Mobil tertua yg dijual keluaran tahun 1982.
* Mayoritas kondisi mobil yg dijual berada di kelas 3-4.
* Mayoritas jarak tempuh mobil yg dijual adalah sebesar 100-1000 km. Mobil yg dijual dgn jarak tempuh terjauh adalah 10000 km.
* Mayoritas harga jual mobil berada pada rentang 5000-15000. Harga tertinggi mobil adalah 235000.

# **Violin plots**

In [ ]:
for i in range(0, len(nums)):
    plt.subplot(1, len(nums), i+1)
    sns.violinplot(y=df[nums[i]], color='gray', orient='v')
    plt.tight_layout()

Alternatif dari kedua plot di atas adalah violin plot. Dengan melihat violin plot di atas, *insight* yang kita temukan ketika melihat box plot dan distribution plot juga dapat diamati.

# **Count plot (categorical)**

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1,3,1)
sns.countplot(df['make'], color='gray', orient='v')
plt.subplot(1,3,2)
sns.countplot(df['model'], color='gray', orient='v')
plt.subplot(1,3,3)
sns.countplot(df['trim'], color='gray', orient='v')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1,2,1)
sns.countplot(df['body'], color='gray', orient='v')
plt.subplot(1,2,2)
sns.countplot(df['transmission'], color='gray', orient='v')
plt.tight_layout()

Berdasarkan count plot diatas,

* untuk kategori transmisi, ditemukan bahwa, mobil dengan transmisi automatic menjadi mobil bekas yang paling dijual

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1,3,1)
sns.countplot(df['state'], color='gray', orient='v')
plt.subplot(1,3,2)
sns.countplot(df['color'], color='gray', orient='v')
plt.subplot(1,3,3)
sns.countplot(df['interior'], color='gray', orient='v')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1,2,1)
sns.countplot(df['seller'], color='gray', orient='v')
plt.subplot(1,2,2)
sns.countplot(df['saledate'], color='gray', orient='v')
plt.tight_layout()

# **Bivariate Analysis**

Setelah menilik setiap kolom secara individual, sekarang kita akan melihat hubungan antar kolom. Mengetahui hubungan antar kolom dapat membantu kita memilih fitur-fitur yang paling penting dan mengesampingkan feature yang redundan.

# **Correlation heatmap**

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(df.corr(), cmap='Blues', annot=True, fmt='.2f')

Dari correlation heatmap di atas dapat dilihat bahwa:

* Target kita sellingprice memiliki korelasi positif kuat dengan tahun
* Memiliki korelasi positif sangat kuat dengan mmr, namun untuk kolom ini tidak akan digunakan maka di stage selanjutnya kolom mmr dapat didrop
* `Condition` memberikan korelasi positif kuat dengan selling price dan mmr.
* `Odometer` memiliki korelasi negatif kuat ke empat fitur lainnya

# **Pair plot**

In [ ]:
plt.figure(figsize=(15, 15))
sns.pairplot(df, diag_kind='kde')

In [ ]:
df.sort_values(['odometer'], ascending=False)

In [ ]:
sns.lineplot(x=df['odometer'],y=df['sellingprice'])

Jika pada plot odometer nilai maks 1 = 382k, maka dari plot tersebut sellingprice akan mulai menunjukan penurunan yang cukup signifikan di odometer dengan nilai 3820

# **Category plots**
Terakhir kita tampilkan *category plot* untuk setiap pasangan kategori *numeric* dan *categorical* seperti di bawah ini

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='transmission', y=nums[i]) # gambar catplot/stripplotnya
    plt.tight_layout() # layout

    if i != 0: plt.close(i+1) # close figure 

    plt.close(len(nums) + 1) # close figure 

Terlihat bahwa jenis transmission memiliki korelasi dengan kolom mmr dan selling price

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)):
    ax = fig.add_subplot(3, 3, i+1)
    sns.stripplot(ax=ax, data=df, x='color', y=nums[i])
    plt.tight_layout()

    if i != 0: plt.close(i+1)

    plt.close(len(nums) + 1)

In [ ]:
for i in range(0, len(nums)):
    ax = fig.add_subplot(3, 3, i+1)
    sns.stripplot(ax=ax, data=df, x='make', y=nums[i])
    plt.tight_layout()

    if i != 0: plt.close(i+1)

    plt.close(len(nums) + 1)

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)):
    ax = fig.add_subplot(3, 3, i+1)
    sns.stripplot(ax=ax, data=df, x='interior', y=nums[i])
    plt.tight_layout()

    if i != 0: plt.close(i+1)

    plt.close(len(nums) + 1)

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)):
    ax = fig.add_subplot(3, 3, i+1)
    sns.stripplot(ax=ax, data=df, x='state', y=nums[i])
    plt.tight_layout()

    if i != 0: plt.close(i+1)

    plt.close(len(nums) + 1)

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)):
    ax = fig.add_subplot(3, 3, i+1)
    sns.stripplot(ax=ax, data=df, x='body', y=nums[i])
    plt.tight_layout()

    if i != 0: plt.close(i+1)

    plt.close(len(nums) + 1)

# **EDA Conclusion**

Beberapa hal yang kita temukan dari EDA dataset ini adalah:

* Data terlihat valid dan tidak ada kecacatan
* Namun masih ada data-data yang kosong/hilang, harus diurus ketika preprocessing
* Ada beberapa distribusi yang sedikit *skewed*, hal ini harus diingat apabila kita ingin melakukan sesuatu atau menggunakan model yang memerlukan asumsi distribusi normal
* Beberapa *feature* memiliki korelasi yang jelas dengan target, mereka akan dipakai
* Beberapa *feature* seperti vin, state, interior, trim, seller, mmr terlihat sama sekali tidak berkorelasi, mereka sebaiknya diabaikan

# **Dataset Overview & Business Insight**

In [ ]:
df['pricesegmentation']=np.where(df['sellingprice']<10000,'low',np.where(df['sellingprice']>30000,'high','med'))
order = ['low', 'med', 'high']
sns.countplot(df['pricesegmentation'],order = order)
plt.xlabel('Price Segmentation',fontsize=12)

Dari segmentasi harga dapat disimpulkan bahwa mayoritas penjualan mobil bekas termasuk dalam kategori medium price.

In [ ]:
jumlah_warna= df.groupby('color',as_index=False).agg(jumlah=('year','count'))

warna_terbesar= jumlah_warna.query('jumlah>10000')

warna_other= pd.Series(jumlah_warna.query('jumlah<=10000').sum())
warna_other.iloc[0]='other'

warna_gabungan= warna_terbesar.append(warna_other, ignore_index=True)
warna_gabungan.sort_values(by=['jumlah'], inplace=True,ascending=False)
plt.bar(data= warna_gabungan, x='color', height='jumlah',color=['black', 'whitesmoke', 'silver', 'gray', 'blue','red','whitesmoke','whitesmoke','green','gold'])
plt.xlabel('Color',fontsize=12)
plt.ylabel('Jumlah',fontsize=12)
nama = ('black', 'white', 'silver', 'gray', 'blue','red','other','unknown','green','gold')
plt.xticks(warna_gabungan['color'], nama)
ax = plt.axes()
ax.set_facecolor('#0198a3')

Dari Bar Chart diatas warna yang memiliki banyak peminatnya yaitu warna hitam dengan total 93245. Lalu, di ikuti oleh warna putih dan silver.

In [ ]:
df['transmission'].value_counts().plot.pie(explode=[0.05, 0.5], autopct='%1.1f%%', figsize=(6,6),fontsize=12)
plt.title('Car Transmission',fontsize=16, fontweight='bold')
plt.ylabel(None)
plt.show()

Dilihat dari grafik transmission ini bahwa sebagian besar dari transmisi adalah automatic

In [ ]:
df=df.dropna()
df.body = df.body.str.lower()
len(df.body.unique())

body_list=[]

for index,kolom in df.iterrows():
  body=kolom['body']
  if body.endswith('cab')==True:
    bodies= 'cab'
  elif body.startswith('cab')==True:
    bodies= 'cab'
  elif body.endswith('coupe')==True:
    bodies= 'coupe'
  elif body.endswith('koup')==True:
    bodies= 'coupe'
  elif body.endswith('van')==True:
    bodies= 'van'
  elif body.endswith('wagon')==True:
    bodies= 'wagon'
  elif body.endswith('convertible')==True:
    bodies= 'convertible'
  elif body.endswith('suv')==True:
    bodies= 'suv'
  elif body.endswith('sedan')==True:
    bodies= 'sedan'
  elif body.endswith('hatchback')==True:
    bodies= 'hatchback'
  else:
    bodies='others'
  body_list.append(bodies)

df['body']=body_list

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y='body', data=df,order=pd.value_counts(df['body']).index)
for i, v in enumerate(pd.value_counts(df['body']).iloc[:9]):
    plt.text(v+100, i, str(v), color='blue')
plt.xlabel('Total of Car',fontsize=12)
plt.ylabel('Body',fontsize=12)
plt.title('Top Used Cars Sold (Body)',fontsize=16, fontweight='bold')
plt.show()

Setelah kolom body distandardize , maka diketahui bahwa konsumen lebih tertarik untuk membeli mobil berjenis sedan jika dibandingkan dengan yang lainnya.

In [ ]:
df.make = df.make.str.lower()

def brand_top10list(x):
  if x['make'] == 'ford':
    brand_top10list = 'ford'
  elif x['make'] == 'chevrolet':
    brand_top10list = 'chevrolet'
  elif x['make'] == 'nissan':
    brand_top10list = 'nissan'
  elif x['make'] == 'toyota':
    brand_top10list = 'toyota'
  elif x['make'] == 'dodge':
    brand_top10list = 'dodge'
  elif x['make'] == 'honda':
    brand_top10list = 'honda'
  elif x['make'] == 'hyundai':
    brand_top10list = 'hyundai'
  elif x['make'] == 'bmw':
    brand_top10list = 'bmw'
  elif x['make'] == 'kia':
    brand_top10list = 'kia'
  elif x['make'] == 'chrysler':
    brand_top10list = 'chrysler'
  elif x['make'] == 'infiniti':
    brand_top10list = 'infiniti'
  elif x['make'] == 'mercedes-benz':
    brand_top10list = 'merce'
  elif x['make'] == 'jeep':
    brand_top10list = 'jeep'
  elif x['make'] == 'volkswagen':
    brand_top10list = 'volkswagen'
  elif x['make'] == 'lexus':
    brand_top10list = 'lexus'
  elif x['make'] == 'gmc':
    brand_top10list = 'gmc'
  elif x['make'] == 'mazda':
    brand_top10list = 'mazda'
  elif x['make'] == 'lincoln':
    brand_top10list = 'lincoln'
  elif x['make'] == 'audi':
    brand_top10list = 'audi'
  else:
    brand_top10list = 'others'
  return brand_top10list

df['top_20_brand'] = df.apply(lambda x: brand_top10list(x), axis=1)

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y='top_20_brand', data=df,order=pd.value_counts(df['top_20_brand']).index)
for i, v in enumerate(pd.value_counts(df['top_20_brand']).iloc[:20]):
    plt.text(v+100, i, str(v), color='blue')
plt.xlabel('Total of Car',fontsize=12)
plt.ylabel('Make',fontsize=12)
plt.title('Top 20 Used Cars Sold (Make)',fontsize=16, fontweight='bold')
plt.show()

* Dengan mengambil 20 penjualan teratas dari kolom make ke kolom top_20_brand, maka akan memasukan beberapa kategori brand(make) yang memiliki nilai jual tinggi dan penjualan yang cukup banyak. Seperti kategori BMW.
* Dari plot histogram, mengambil 20 brand dengan penjualan terbanyak untuk stage selanjutnya dirasa cukup sesuai, karena jika diambil 10 brand maka akan ada beberapa penjualan yang cukup laris dan memiliki range price low - high masuk kedalam kategori others

In [ ]:
top_make=df['make'].value_counts()[:10].index
fig,ax=plt.subplots(5,2,figsize=(14,20))
color_list=['#0a9396','#ca6702','#ae2012','#9b2226','#001219','#005f73','#94d2bd','#e9d8a6','#e5e5e5','#e07a5f'] #coolors.co
i=0
for t in top_make:
    df.loc[df['make']==t,'sellingprice'].hist(ax=ax[i%5][i//5],bins=100,color=np.random.choice(color_list,replace=False))
    ax[i%5][i//5].set_xlabel('Selling Price',fontsize=10)
    ax[i%5][i//5].set_ylabel('Frequency',fontsize=10)
    ax[i%5][i//5].set_title(f'Selling price distribution for {t}',fontsize=15)
    plt.subplots_adjust(hspace=0.45)
    i+=1

In [ ]:
graph = sns.FacetGrid(df, col ="body",col_wrap=3)
graph.map(plt.scatter, "odometer", "sellingprice", edgecolor ="w").add_legend()
plt.show()

In [ ]:
graph = sns.FacetGrid(df, col ="transmission")
graph.map(plt.scatter, "odometer", "sellingprice", edgecolor ="w").add_legend()
plt.show()


In [ ]:
graph = sns.FacetGrid(df, col ="top_20_brand",col_wrap=3)
graph.map(plt.scatter, "odometer", "sellingprice", edgecolor ="w").add_legend()
plt.show()

In [ ]:
def drop_val(col):
    idx_to_drop = []
    for i in col.index:
        idx_to_drop.append(i)

    df.drop(idx_to_drop,inplace=True)
    df.reset_index(drop=True,inplace=True)
    
fixed_date = []
for i in range(len(df['saledate'])):
    date = df.saledate[i][:15]
    fixed_date.append(date)
    
df['saledate'] = fixed_date
df['saledate'] = pd.to_datetime(df['saledate'])
date_not_use = df[(df['saledate'] < '2014-12-01')]  # Deleting this data because there aren't enough data for that range of date 
drop_val(date_not_use)    

In [ ]:
median_saledate = df.groupby(df['saledate'].dt.strftime('%Y-%m')).median()
plt.figure(figsize=(18,6))
plt.title('Selling Price of Used Car by Month of Year',fontsize=16, fontweight='bold')
sns.pointplot(x=median_saledate.index,y=median_saledate['sellingprice'], data=median_saledate,ci=None)
for i, v in enumerate(median_saledate['sellingprice']):
    if v >= np.median(median_saledate['sellingprice']):
        plt.text(i-0.1, v+500, str(v), color='green', fontweight='bold')
    else:
        plt.text(i-0.1, v-1000, str(v), color='red', fontweight='bold')
                    
plt.axhline(np.median(median_saledate['sellingprice']),color='blue', linestyle='--', label='median')
plt.ylabel ('Selling Price',fontsize=12)
plt.xlabel('Sale Month',fontsize=12)
plt.ylim(0,20000)
plt.legend()
plt.show()

In [ ]:
yr_gb = df.groupby(df['saledate'].dt.strftime('%Y-%m')).count()
plt.figure(figsize=(18,6))
plt.title('Amount Used Car Sold by Month of Year',fontsize=16, fontweight='bold')
sns.pointplot(x=yr_gb.index,y=yr_gb['make'], data=yr_gb,ci=None)
for i, v in enumerate(yr_gb['make']):
    if v >= np.mean(yr_gb['make']):
        plt.text(i-0.1, v+5000, str(v), color='green', fontweight='bold')
    else:
        plt.text(i-0.1, v+5000, str(v), color='red', fontweight='bold')
                    
plt.axhline(np.mean(yr_gb['make']),color='blue', linestyle='--', label='mean')
plt.ylabel ('Total Number of Transaction',fontsize=12)
plt.xlabel('Sale Month',fontsize=12)
plt.ylim(0,200000)
plt.legend()
plt.show()

In [ ]:
df['combined'] = df['make'].str.cat(df[['model','trim']].astype(str), sep=' ')
plt.figure(figsize=(14,8))
sns.countplot(y='combined',data=df,order=pd.value_counts(df['combined']).iloc[:10].index)
for i, v in enumerate(pd.value_counts(df['combined']).iloc[:10]):
    plt.text(v+100, i, str(v), color='blue')
plt.xlabel('Total of Car',fontsize=12)
plt.ylabel('Car',fontsize=12)
plt.title('Top 10 Used Cars Sold',fontsize=16, fontweight='bold')
plt.show()

In [ ]:
df_sort=df.sort_values(by='sellingprice',ascending=False,ignore_index=True)
df_sort=df_sort.iloc[1:20]
plt.figure(figsize=(14,8))
sns.barplot(y='combined',x='sellingprice',data=df_sort)
plt.xlabel('Price',fontsize=12)
plt.ylabel('Car',fontsize=12)
plt.title('Top 10 Most Expensive Car',fontsize=16, fontweight='bold')
plt.show()

In [ ]:
df_sort=df.sort_values(by='sellingprice',ascending=True,ignore_index=True)
df_sort=df_sort[~(df_sort['sellingprice'] < 1300) ]
df_sort=df_sort.iloc[:10]
plt.figure(figsize=(14,8))
sns.barplot(y='combined',x='sellingprice',data=df_sort)
plt.xlabel('Price',fontsize=12)
plt.ylabel('Car',fontsize=12)
plt.title('Top 10 Cheapest Car',fontsize=16, fontweight='bold')
plt.show()

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(10,6))
sns.barplot(df['color'].value_counts().iloc[:5].index,df['color'].value_counts().iloc[:5],color='lightgreen', ax=ax[0])
for i, v in enumerate(df['color'].value_counts().iloc[:5]):
    ax[0].text(i-0.15, v+1000, str(v), color='brown', fontweight='bold')
sns.barplot(df['interior'].value_counts().iloc[:5].index,df['interior'].value_counts().iloc[:5],color='lightblue', ax=ax[1])
for i, v in enumerate(df['interior'].value_counts().iloc[:5]):
    ax[1].text(i-0.15, v+1000, str(v), color='brown', fontweight='bold')
ax[0].set_title('Top 5 Number of Body Car Colors',fontsize = 16, fontweight='bold')
ax[0].set_xlabel('Colors',fontsize = 12)
ax[0].set_ylabel(None)
ax[1].set_title('Top 5 Number of Interior Car Colors',fontsize = 16, fontweight='bold')
ax[1].set_xlabel('Colors',fontsize = 12)
ax[1].set_ylabel(None)
ax[0].set_ylim(0,250000)
ax[1].set_ylim(0,250000)
fig.show()

plt.subplots_adjust(right = 1.5)

In [ ]:
df_clean=df.dropna()
df_groupby=df_clean.groupby(by='year').mean().reset_index()
sns.barplot(x='year',y='sellingprice',data=df_groupby)
plt.xticks(rotation=90)
plt.xlabel('Year',fontsize=12)
plt.ylabel('Price',fontsize=12)
plt.title('Average Used Car Price per Year',fontsize=16, fontweight='bold')
plt.show();

In [ ]:
dfbody = pd.DataFrame({'Percentage': [39,22,6,5,4,24]},
                  index=['Sedan', 'SUV',
                              'Cab', 'Van','Hatchback','Other'])
dfbody.plot.pie(y='Percentage', autopct='%1.0f%%', figsize=(8,8),fontsize=12)
plt.title('Top 5 Body',fontsize=16, fontweight='bold')
plt.ylabel(None)
plt.show()

In [ ]:
dfmake = pd.DataFrame({'Percentage': [17,11,9,7,6,50]},
                  index=['Ford', 'Chevrolet',
                              'Nissan', 'Toyota','Dodge','Other'])
dfmake.plot.pie(y='Percentage', autopct='%1.0f%%', figsize=(8,8),fontsize=12)
plt.title('Top 5 Make',fontsize=16, fontweight='bold')
plt.ylabel(None)
plt.show()

In [ ]:
dfcolor = pd.DataFrame({'Percentage': [20,19,15,14,9,23]},
                  index=['Black', 'White',
                              'Silver', 'Gray','Blue','Other'])
colors = ['#696969', '#FFFFFF', '#C0C0C0', '#d3d3d3','#1e90ff','#EADDCA']
dfcolor.plot.pie(y='Percentage', autopct='%1.0f%%', figsize=(8,8),fontsize=12,colors=colors)
plt.title('Top 5 Color',fontsize=16, fontweight='bold')
plt.ylabel(None)
plt.show()

**Stage 2 Final Project - Data Pre-processing**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns